## BBM 444 - Programming Assignment 1: Camera Pipeline

* You can add as many cells as you want in-between each question.
* Please add comments to your code to explain your work.  
* Please add Markdown cells to answer the (non-coding) questions in the homework text. You can, however, refer to the outputs of code cells without adding them as images to the Markdown cell unless you are requested to do otherwise.
* Please be careful about the order of runs of cells. Doing the homework, it is likely that you will be running the cells in different orders, however, they will be evaluated in the order they appear. Hence, please try running the cells in this order before submission to make sure they work.    
* Please refer to the homework text for any implementation detail. Though you are somewhat expected to abide by the comments in the below cells, they are mainly just provided for guidance. That is, as long as you are not completely off this structure and your work pattern is understandable and traceable, it is fine. For instance, you do not have to implement a particular function within a cell just because the comment directs you to do so.
* This document is also your report. Show your work.

##  Insert personal information (name, surname, student id)
Bedirhan Gençaslan
2210356065

## 1. Implement a basic image processing pipeline (80 points)

### 1.1 Raw Image Conversion (5 points)

In [694]:
## Initialize the variables obtained with "dcraw -4 -d -v -T sample.NEF"


black = 0
white = 4095

r_scale = 2.629696
g_scale = 1.000000
b_scale = 1.251207

### 1.2. Python Initials (5 points)

In [695]:
## Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

In [696]:
## Read the image
image = imread('sample.tiff')

# Get dimensions
height, width = image.shape[:2]
print(f"Image shape: {image.shape}")
print(f"Image dtype: {image.dtype}")

# Calculate bit depth
bit_depth = image.dtype.itemsize * 8

# Print results
print(f"Width: {width}, Height: {height}, Bits per pixel: {bit_depth}")


Image shape: (2014, 3039)
Image dtype: uint16
Width: 3039, Height: 2014, Bits per pixel: 16


In [697]:
## Convert the image to double-precision array
image_double = image.astype(np.float64)


print(f"Original dtype: {image.dtype}, New dtype: {image_double.dtype}")



Original dtype: uint16, New dtype: float64


### 1.3. Linearization (5 points)

In [698]:
## Apply a linear transformation so that the value <black> is mapped to 0 and <white> to 1. Then, clip negative values to 0
## and values greater than 1 to 1. (See numpy function clip.)
## ( <black> and <white> values are obtained in the first step, raw image conversion ).
## Linear transformation to map black to 0 and white to 1, then clip
image_linear = np.clip((image_double - black) / (white - black), 0, 1)

print(f"Linearized image shape: {image_linear.shape}")

# Print min and max values to verify
print(f"Min: {image_linear.min()}, Max: {image_linear.max()}")


Linearized image shape: (2014, 3039)
Min: 0.003663003663003663, Max: 1.0


### 1.4. White Balancing (10 points)

#### In the following 3 cells you are asked to implement three different automatic white balancing algorithms. The parameters for these functions are up to you. However, please comment, stating their types and purposes.

In [699]:
def white_balance_white_world(image):
    # Compute per-channel maximum values (R, G, B)
    r_max, g_max, b_max = np.max(image, axis=(0, 1))
    # Normalize each channel by its maximum, then scale by the green channel maximum
    balanced = np.empty_like(image)
    balanced[..., 0] = image[..., 0] / r_max * g_max
    balanced[..., 1] = image[..., 1]
    balanced[..., 2] = image[..., 2] / b_max * g_max
    return np.clip(balanced, 0, 1)

In [700]:
## Implement a function for grey world assumption automatic white balancing algorithm.
def white_balance_gray_world(image):
    # Compute per-channel averages (R, G, B)
    r_avg, g_avg, b_avg = np.mean(image, axis=(0, 1))
    # Normalize each channel by its own average, then scale by the green channel average
    balanced = np.zeros_like(image)
    balanced[..., 0] = image[..., 0] / r_avg * g_avg
    balanced[..., 1] = image[..., 1]
    balanced[..., 2] = image[..., 2] / b_avg * g_avg
    return np.clip(balanced, 0, 1)

In [701]:
## Implement a function for a white balancing algorithm, where the red, green, and blue channels are multiplied with
## <r scale>, <g scale>, and <b scale> recorded in the first step, which is the white balancing preset used by the camera.
def white_balance_preset(image, r_scale, g_scale, b_scale):
    # Create an array of the provided scale factors and multiply per-channel.
    scales = np.array([r_scale, g_scale, b_scale])
    balanced = image * scales
    return np.clip(balanced, 0, 1)

### 1.5. Experiment (possibly with white balancing and/or demosaicing) to identify the correct Bayer Pattern. State which pattern you idenfitied with your logic behind in a seperate markdown cell. (20 points)
#### For demosaicing, apply bilinear interpolation using scipy’s built-in interp2d function.
#### Display each resulting image of your experiments, stating the configuration of the bayyer pattern and the other methods you use, such as WA algorithm.

In [702]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator

import numpy as np
from scipy.interpolate import RegularGridInterpolator

def bayer(image, pattern='rggb'):
    # Ensure even dimensions
    h, w = image.shape
    if h % 2: image = image[:-1, :]
    if w % 2: image = image[:, :-1]
    h, w = image.shape

    # Split into 2x2 sub-arrays
    im1 = image[0::2, 0::2] 
    im2 = image[0::2, 1::2] 
    im3 = image[1::2, 0::2] 
    im4 = image[1::2, 1::2] 

    # Map each pattern to its channel sub-arrays and their grid offsets
    patterns = {
        'rggb': {'R': (im1, 0, 0), 'GU': (im2, 0, 1), 'GD': (im3, 1, 0), 'B': (im4, 1, 1)},
        'grbg': {'R': (im2, 0, 1), 'GU': (im1, 0, 0), 'GD': (im4, 1, 1), 'B': (im3, 1, 0)},
        'bggr': {'R': (im4, 1, 1), 'GU': (im3, 1, 0), 'GD': (im2, 0, 1), 'B': (im1, 0, 0)},
        'gbrg': {'R': (im3, 1, 0), 'GU': (im1, 0, 0), 'GD': (im4, 1, 1), 'B': (im2, 0, 1)}
    }
    if pattern not in patterns:
        raise ValueError("Invalid Bayer pattern. Choose from 'rggb', 'grbg', 'bggr', or 'gbrg'.")

    # Helper function to interpolate a sub-array to full resolution.
    def interp_channel(sub, row_offset, col_offset):
        y_known = np.arange(row_offset, h, 2)
        x_known = np.arange(col_offset, w, 2)
        interp = RegularGridInterpolator((y_known, x_known), sub,
                                         method='linear', bounds_error=False, fill_value=0)
        grid_y, grid_x = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
        return interp(np.stack([grid_y, grid_x], axis=-1))
    
    # Interpolate channels
    R = interp_channel(*patterns[pattern]['R'])
    G = (interp_channel(*patterns[pattern]['GU']) + interp_channel(*patterns[pattern]['GD'])) / 2
    B = interp_channel(*patterns[pattern]['B'])
    
    rgb = np.stack((R, G, B), axis=-1)
    rgb = np.clip(rgb, 0, None)
    return rgb


In [ ]:
## Display the resulting images using "each" white balancing algorithm you implemented above.

def demosaic_for_allcases(image_linear , pattern = 'bggr'):# Generate different Bayer pattern images    

    bayer_image = bayer(image_linear, pattern=pattern)
    white_world = white_balance_white_world(bayer_image)
    gray_world = white_balance_gray_world(bayer_image)
    scaled = white_balance_preset(bayer_image, r_scale, g_scale, b_scale)


    plt.figure(figsize=(12 , 6))
    plt.subplot(1, 3, 1)
    plt.title("White world balance")
    plt.imshow(white_world)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Gray world balance")
    plt.imshow(gray_world)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Camera preset balance")
    plt.imshow(scaled)
    plt.axis('off')


    plt.show()
    
demosaic_for_allcases(image_linear , 'bggr')
demosaic_for_allcases(image_linear , 'rggb')
demosaic_for_allcases(image_linear , 'grbg')
demosaic_for_allcases(image_linear , 'gbrg')


bayer_image = bayer(image_linear, pattern='bggr')
white_world = white_balance_white_world(bayer_image)
gray_world = white_balance_gray_world(bayer_image)
scaled = white_balance_preset(bayer_image, r_scale, g_scale, b_scale)

#### HERE REPORT THE IDENTIFIED BAYER PATTERN AND YOUR LOGIC

There is a important difference between these 4 images. Below the colors are different . I looked to original image and there is a orange under brown starting the table and only a image like original that is BGGR filter image.

### 1.6. Color space correction. (10 points)
#### Transform your image to the linear sRGB color space. For this, you need to compute $M_{cam\rightarrow sRGB}$ matrix as explained in the homework text.

In [704]:
# Camera multipliers obtained from Dcraw output (Example values)
import numpy as np
""" "Nikon D40", 0, 0,
        { 6992,-1668,-806,-8138,15748,2543,-874,850,7897 } },"""
# Example camera multipliers obtained from Dcraw output (R, G, B)
camera_multipliers = np.array([2.629696, 1.000000, 1.251207])
# Normalize multipliers so that the green channel is 1.0
camera_multipliers /= camera_multipliers[1]

# XYZ → Camera transformation matrix (example from Nikon D40 in dcraw)
M_XYZ_to_cam = np.array([[6992, -1668, -806],
                           [-8138, 15748, 2543],
                           [-874, 850, 7897]])
M_XYZ_to_cam = M_XYZ_to_cam / 10000.0
# sRGB → XYZ transformation matrix (constant as per sRGB standard)
M_sRGB_to_XYZ = np.array([
    [0.4124564, 0.3575761, 0.1804375],
    [0.2126729, 0.7151522, 0.0721750],
    [0.0193339, 0.1191920, 0.9503041]
])

# Scale the camera matrix with the multipliers (broadcasting over columns)
M_XYZ_to_cam *= camera_multipliers[:, np.newaxis]

# Compute forward transformation (sRGB → camera) using Equation (2)
M_sRGB_to_cam = M_XYZ_to_cam @ M_sRGB_to_XYZ

# Normalize the forward matrix so that each row sums to 1 (ensuring white remains white)
M_sRGB_to_cam_normalized = M_sRGB_to_cam / M_sRGB_to_cam.sum(axis=1, keepdims=True)

# Invert the normalized matrix to obtain the camera-to-sRGB transformation matrix (Equation 1)
M_cam_to_sRGB = np.linalg.inv(M_sRGB_to_cam_normalized)



In [705]:
def apply_color_space_correction(image_rgb, M_cam_to_sRGB):
    height, width, _ = image_rgb.shape
    # Reshape image to (num_pixels, 3) to apply the matrix transformation to each pixel
    pixels = image_rgb.reshape(-1, 3)
    # Apply transformation (each pixel: [R_sRGB, G_sRGB, B_sRGB] = M_cam_to_sRGB · [R_cam, G_cam, B_cam])
    corrected_pixels = pixels @ M_cam_to_sRGB.T
    # Clip values to ensure they remain within the [0, 1] range
    corrected_pixels = np.clip(corrected_pixels, 0, 1)
    # Reshape back to the original image dimensions
    corrected_image = corrected_pixels.reshape(height, width, 3)
    return corrected_image


In [ ]:
# Apply color transformation (Camera RGB → sRGB)
transformed_image1 = apply_color_space_correction(white_world, M_cam_to_sRGB)
transformed_image2 = apply_color_space_correction(gray_world, M_cam_to_sRGB)
transformed_image3 = apply_color_space_correction(scaled, M_cam_to_sRGB)

plt.figure(figsize=(12 , 6))
plt.subplot(1, 3, 1)
plt.title("White world balance")
plt.imshow(transformed_image1)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Gray world balance")
plt.imshow(transformed_image2)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Camera preset balance")
plt.imshow(transformed_image3)
plt.axis('off')


plt.show()


### 1.7. Brightness adjustment and gamma encoding. (10 points)

In [ ]:
from skimage import color

import numpy as np
import matplotlib.pyplot as plt
from skimage import color

def adjust_brightness(image, target_mean_intensity=0.25):
    # 1. Convert the image to grayscale
    grayscale_image = color.rgb2gray(image)
    
    # 2. Compute the current mean intensity
    current_mean_intensity = np.mean(grayscale_image)
    
    # 3. Calculate the scaling factor to reach the target mean intensity
    scale_factor = target_mean_intensity / current_mean_intensity
    
    # 4. Scale the image brightness
    brightened_image = image * scale_factor
    
    # 5. Clip pixel values to ensure they remain within [0,1]
    brightened_image = np.clip(brightened_image, 0, 1)
    
    return brightened_image

def gamma_encode(image):
    # Apply gamma correction:
    # If C_linear <= 0.0031308 -> 12.92 * C_linear
    # If C_linear > 0.0031308 -> (1 + 0.055) * C_linear^(1/2.4) - 0.055
    
    image_gamma_encoded = np.where(image <= 0.0031308, 
                                   12.92 * image, 
                                   (1 + 0.055) * np.power(image, 1/2.4) - 0.055)
    
    return image_gamma_encoded

def experiment_with_scaling(image, scale_values):
    fig, axes = plt.subplots(2, len(scale_values), figsize=(20, 10))  # 2 rows for comparison
    
    for i, scale in enumerate(scale_values):
        # 1. Adjust brightness
        brightened_image = adjust_brightness(image, target_mean_intensity=scale)
        
        # 2. Apply gamma correction
        gamma_corrected_image = gamma_encode(brightened_image)
        
        # 3. First row: Brightness-adjusted image
        axes[0, i].imshow(brightened_image)
        axes[0, i].set_title(f'Brightened (Scale: {scale})')
        axes[0, i].axis('off')

        # 4. Second row: Gamma-corrected image
        axes[1, i].imshow(gamma_corrected_image)
        axes[1, i].set_title(f'Gamma Corrected (Scale: {scale})')
        axes[1, i].axis('off')
    
    plt.show()

def process_image(image, target_mean=0.25):
    # Step 1: Adjust brightness
    brightened_image = adjust_brightness(image, target_mean)
    
    # Step 2: Apply gamma encoding
    final_image = gamma_encode(brightened_image)
    
    return final_image

# Example usage
scale_values = [0.1, 0.25, 0.5, 0.75, 1.0]
experiment_with_scaling(transformed_image2, scale_values)

# Normalize the image by dividing by 65535 (assuming a 16-bit image)
corrected_image = transformed_image2 / 65535.0

processed_image = process_image(corrected_image, target_mean=0.25)


#### Report your choice for scaling value here

In [708]:
"""0.25 is the best ratio for this example to human eyes because it doesn't lose colors in the picture."""

"0.25 is the best ratio for this example to human eyes because it doesn't lose colors in the picture."

### 1.8. Compression (5 points)

In [ ]:
## Use skimage's imsave function to store the image in .png format (no compression), and also in .jpeg format with the quality
## parameter set to 95
from skimage.io import imsave
import numpy as np
import os  # Ensure os module is imported
import imageio  # Import imageio to save images
from skimage.io import imsave

import numpy as np
import imageio
import os

def save_and_compute_compression(image, filename_base="output_image"):
    """
    Saves an image in both PNG (lossless) and JPEG (quality=95) formats and computes the compression ratio.
    
    Parameters:
    - image: Input image (normalized in range [0,1]).
    - filename_base: Base name for the output files.
    
    Returns:
    - png_filename: Saved PNG file name.
    - jpeg_filename: Saved JPEG file name.
    - compression_ratio: Ratio of PNG file size to JPEG file size.
    """
    # Convert image to uint8 format (scale values to [0, 255])
    image = (image * 255).astype(np.uint8)
    
    # Save as PNG (lossless format)
    png_filename = filename_base + ".png"
    imageio.imwrite(png_filename, image)

    # Save as JPEG with quality=95
    jpeg_filename = filename_base + "_q95.jpg"
    imageio.imwrite(jpeg_filename, image, quality=95)

    # Get file sizes
    png_size = os.path.getsize(png_filename)
    jpeg_size = os.path.getsize(jpeg_filename)

    # Compute compression ratio (PNG / JPEG)
    compression_ratio = png_size / jpeg_size
    print(f"PNG size: {png_size} bytes")
    print(f"JPEG (quality 95) size: {jpeg_size} bytes")
    print(f"Compression ratio: {compression_ratio:.2f}")

    return png_filename, jpeg_filename, compression_ratio

# Example usage
png_file, jpeg_file, ratio = save_and_compute_compression(processed_image)



In [ ]:
## Compute the compression ratio
import os

def find_best_quality(image, filename_base="output_image"):
    for quality in range(95, 45, -15):  # 95'ten 50'ye kadar 5'er azalarak dene
        jpeg_filename = f"{filename_base}_q{quality}.jpg"
        
        # Normalize the image to the range [0, 255] and convert to uint8
        image_uint8 = np.clip(image * 255.0, 0, 255).astype(np.uint8)
        
        # Save the image with the given quality
        imsave(jpeg_filename, image_uint8, quality=quality, check_contrast=False)

        # Get file size
        jpeg_size = os.path.getsize(jpeg_filename)
        print(f"Kalite {quality}: {jpeg_size} byte")

# Kullanım
find_best_quality(processed_image)


#### Comment on the difference between files and state the compression ratio

In [711]:
## Show the images with different JPEG quality settings to determine the lowest setting for which the compressed image is
## indistinguishable from the original.

"""65 is best compression ratio it can't distinguishable with 95 and size of 65 very small according to 95."""

"65 is best compression ratio it can't distinguishable with 95 and size of 65 very small according to 95."

#### State the quality setting you have chosen and the corresponding compression ratio.

## 2.Perform manual white balancing (10 points)

#### See matplotlib function ginput to select image coordinates interactively.
#### Select some pathces in the scene you expect to be white and experiment on them appling the explained in the homework text. Experimenting with a few pathces suffices.

In [712]:
## Select patches in the scene you expect to be white. (Hint: Use ginput)

from matplotlib.path import Path
import numpy as np
from matplotlib.path import Path

def manual_white_balance(img_rgb, corners):
    # Create a Path object from the given rectangle coordinates
    path = Path(corners)
    
    # Get the image dimensions
    h, w, _ = img_rgb.shape
    
    # Create a grid of pixel coordinates
    x, y = np.meshgrid(np.arange(w), np.arange(h))
    points = np.vstack((x.flatten(), y.flatten())).T
    
    # Check which pixels are inside the given region
    mask = path.contains_points(points)
    mask = mask.reshape(h, w)
    
    # Extract the pixels inside the selected region
    patch = img_rgb[mask]
    
    # If no pixels are selected, return the original image
    if patch.size == 0:
        print("No pixels found in the selected region.")
        return img_rgb.copy()
    
    # Compute the maximum intensity values for each channel (R, G, B)
    R_avg = np.mean(patch[:, 0])
    G_avg = np.mean(patch[:, 1])
    B_avg = np.mean(patch[:, 2])
    
    # If any channel has zero intensity, return the original image
    if R_avg == 0 or G_avg == 0 or B_avg == 0:
        print("One of the channels has zero intensity in the selected region.")
        return img_rgb.copy()
    
    # Compute scaling factors
    scale_R = 1 / R_avg
    scale_G = 1 / G_avg
    scale_B = 1 / B_avg
    
    # Apply scaling to adjust white balance
    img_balanced = np.zeros_like(img_rgb)
    img_balanced[:, :, 0] = img_rgb[:, :, 0] * scale_R
    img_balanced[:, :, 1] = img_rgb[:, :, 1] * scale_G
    img_balanced[:, :, 2] = img_rgb[:, :, 2] * scale_B
    
    # Clip pixel values to ensure they remain within [0, 1] range
    img_balanced = np.clip(img_balanced, 0, 1)
    
    return img_balanced


In [713]:
# Load the image using PIL
from PIL import Image
import matplotlib.pyplot as plt

In [714]:
## Implement the algorithm in the HW text possibly with function(s)

#### Experiment with patch 1

In [ ]:
# Load and prepare the image
image_path = 'output_image_q65.jpg'  # Set the image file path
img = Image.open(image_path)  # Open the image using PIL
img_rgb = np.array(img).astype(np.float64) / 255.0  # Normalize pixel values to [0, 1]

print("Image shape:", img_rgb.shape)  # Print the shape of the image

# Define the rectangular region's corner coordinates (example)
corners = [(2410, 1830), (2412, 1830), (2410, 1832), (2412, 1832)]  # Coordinates of the white region in the table

# Apply white balance
img_balanced1 = manual_white_balance(img_rgb, corners)  # Perform white balance on the image

# Display the original and balanced images side by side
plt.figure(figsize=(10, 5))  # Set figure size for the display

plt.subplot(1, 2, 1)  # Create the first subplot
plt.imshow(img_rgb)  # Display the original image
plt.title('Original Image')  # Title for the original image
plt.axis('off')  # Turn off axis for better visualization

plt.subplot(1, 2, 2)  # Create the second subplot
plt.imshow(img_balanced1)  # Display the white-balanced image
plt.title('White-Balanced Image')  # Title for the white-balanced image
plt.axis('off')  # Turn off axis for better visualization

plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()  # Show the images



In [716]:
## Show the corresponding resulting image after applying the necessary steps needed to get a properly displayable image.

#### Experiment with patch 2

In [ ]:
## Normalize all three channels using weights that make the R, G, B channel values of this patch equal
# Load and prepare the image
image_path = 'output_image_q65.jpg'  # Specify the image file path here
img = Image.open(image_path)  # Open the image using PIL
img_rgb = np.array(img).astype(np.float64) / 255.0  # Normalize pixel values to [0, 1]

print(img_rgb.shape)  # Print the shape of the image

# Define the corner coordinates of the rectangle (example)
corners = [(1000, 1500), (1000, 2000), (2000, 1500), (2000, 2000)]  # Rectangle corners' coordinates

# Apply white balance processing
img_balanced2 = manual_white_balance(img_rgb, corners)  # Perform white balance on the image

# Display the original and balanced images side by side
plt.figure(figsize=(10, 5))  # Set figure size for the display

plt.subplot(1, 2, 1)  # Create the first subplot
plt.imshow(img_rgb)  # Display the original image
plt.title('Original Image')  # Title for the original image
plt.axis('off')  # Turn off axis for better visualization

plt.subplot(1, 2, 2)  # Create the second subplot
plt.imshow(img_balanced2)  # Display the white-balanced image
plt.title('Balanced Image')  # Title for the white-balanced image
plt.axis('off')  # Turn off axis for better visualization

plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()  # Show the images

In [718]:
pngfile, jpegfile , _  = save_and_compute_compression(img_balanced1 , "final_image")

PNG size: 5872341 bytes
JPEG (quality 95) size: 1160183 bytes
Compression ratio: 5.06


In upper image I picked coordinates on white side in table. In second image I pick big area of image. So in the first part algorithm ran true way but in the second part algorithm try to convert darker sides to white in image that's makes whiter than first.

## 3. Learn to use dcraw (10 points)

#### Reading the dcraw documentation, report the correct values for flags for the image processing pipeline you have implemented here.

#### You have at least three different images, namely, (1) at least one produced in the image processing pipeline you implemented, (2) produced using dcraw, and (3) image provided in the assignment data folder. Compare, explain the differences among them, and state your favorite one.
* Please add these images here. For this, you can use the following:
```python
from IPython.display import Image
```

In [ ]:
from IPython.display import Image, display

display(Image(filename="dcraw_result.png"))
display(Image(filename="sample.jpg"))
display(Image(filename="final_image.png"))

sample.jpg is best. dcraw_result.png made by camera presets but that broke it's colors. The last image has bad resolution.